In [24]:
import numpy as np
import networkx as nx
import os
import osmnx as ox
import time


# Call this function to get the data 
def get_data(city='Detroit', mode='bike', root_path='../data/'):
    folders = os.listdir(root_path)
    for folder in folders:
        if folder == city:
            files = os.listdir(root_path+folder)
            bike_data = [file for file in files if mode in file][0]
            path = root_path + folder+ '/'+ bike_data;
    return ox.load_graphml('C:/Users/nlepore/ComplexityScienceProject1/data/{}/{}_bike.graphml'.format(name,name))

    #return nx.read_graphml(path)

G = get_data()
print(type(G))

<class 'networkx.classes.multidigraph.MultiDiGraph'>


#### Define helper functions to compute metrics

In [2]:
def convert_lengths_to_floats(G):
    """
    Changes the attributes of the edges of graph G such that the 'length' field
    is a float rather than a string
    """
    for u, v, d in list(G.edges(data=True)):
        d['length'] = float(d['length'])

def get_edge_length(G, u, v):
    """
    given two nodes in graph G, return the straight line distance 
    (as the crow flies) between them
    """
    xdist = float(G.nodes[u]['x']) - float(G.nodes[v]['x'])
    ydist = float(G.nodes[u]['y']) - float(G.nodes[v]['y'])
    length = (xdist**2 + ydist**2)**0.5
    return length

def make_new_random_edge(G, u=None):
    """
    Return a tuple of nodes representing a random new and unique edge
    """
    if not u:
        u = np.random.choice(G.nodes(), 1)
        u = u[0]
    
    available = [node for node in G.nodes() if node not in G[u]]
    
    # choose v if not specified
    v = np.random.choice(available, 1)
    v = v[0]
    return (u, v)

    
def calc_directness(G, u, v):
    """
    Calculate the directness between two nodes u and v
    Directness is the ratio between:
        - the shortest node path length (number of edges times the length of each edge)
        - the straight line distance between those two nodes
    """
    path_length = nx.shortest_path_length(G, source=u, target=v, weight='length')
    direct_length = get_edge_length(G, u, v)
    return direct_length/path_length

def calc_avg_directness(G, n):
    """
        Given a weakly connected directed graph component, G
        Pick n pairs of random nodes, and caluclate the average directness
    """
    d = 0
    for _ in range(n):
        start = np.random.choice(G.nodes(), 1)
        start = start[0]
        reachable = nx.descendants(G, start)
        if not reachable:
            continue
        end = np.random.choice(list(reachable), 1)
        end = end[0]

        d += (calc_directness(G, start, end) / n);
    return d

def euclidean_dist_vec(y1, x1, y2, x2):
    '''
    Calculate the euclidean distance between two points.
    '''
    distance = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
    return distance


#### Define Methods for connecting the Graphs

In [4]:
def L2S(wcc):
    '''
    Find the closest pair of nodes between two different connected components.
    ---
    wcc: list connected components

    returns: dict nodes i and j and distance
    '''
    closest_pair = {'i': 0, 'j': 0, 'dist': np.inf}
    for i in wcc[0].nodes(data=True):
        i_coord = (i[1]['y'], i[1]['x'])
        for j in wcc[1].nodes(data=True):
            j_coord = (j[1]['y'], j[1]['x'])
            dist = euclidean_dist_vec(i_coord[0], i_coord[1], j_coord[0], j_coord[1])
            if dist < closest_pair['dist']:
                closest_pair['i'] = i[0]
                closest_pair['j'] = j[0]
                closest_pair['dist'] = dist
    return closest_pair

def L2C(wcc):
    closest_pair = {'i': 0, 'j': 0, 'dist': np.inf}
    for i in wcc[0].nodes(data=True):
        i_coord = (i[1]['y'], i[1]['x'])
        for j in wcc[1:]:
            for k in j.nodes(data=True):
                j_coord = (k[1]['y'], k[1]['x'])
                dist = euclidean_dist_vec(float(i_coord[0]), float(i_coord[1]), float(j_coord[0]), float(j_coord[1]))
                if dist < closest_pair['dist']:
                    closest_pair['i'] = i[0]
                    closest_pair['j'] = k[0]
                    closest_pair['dist'] = dist
    return closest_pair

def R2C(wcc):
    '''
    Find the closest pair of nodes between two different connected components.
    ---
    wcc: list connected components

    returns: dict nodes i and j and distance
    '''
    closest_pair = {'i': 0, 'j': 0, 'dist': np.inf}
    num_clusters = len(wcc)
    cluster = np.random.choice(len(wcc))
    for i in wcc[cluster].nodes(data=True):
        i_coord = (i[1]['y'], i[1]['x'])
        for w,j in enumerate(wcc[1:]):
            if w == cluster - 1:
                break
            for k in j.nodes(data=True):
                j_coord = (k[1]['y'], k[1]['x'])
                dist = euclidean_dist_vec(float(i_coord[0]), float(i_coord[1]), float(j_coord[0]), float(j_coord[1]))
                if dist < closest_pair['dist']:
                    closest_pair['i'] = i[0]
                    closest_pair['j'] = k[0]
                    closest_pair['dist'] = dist
    return closest_pair

def Closest(wcc):
    closest_pair = {'i': 0, 'j': 0, 'dist': np.inf}
    for v,i in enumerate(wcc[0:]):
            for u in i.nodes(data=True):
                i_coord = (u[1]['y'], u[1]['x'])
                for w,j in enumerate(wcc[0:]):
                    if w == v:
                        break
                    for k in j.nodes(data=True):
                        j_coord = (k[1]['y'], k[1]['x'])
                        dist = euclidean_dist_vec(float(i_coord[0]), float(i_coord[1]), float(j_coord[0]), float(j_coord[1]))
                        if dist < closest_pair['dist']:
                            closest_pair['i'] = u[0]
                            closest_pair['j'] = k[0]
                            closest_pair['dist'] = dist
    return closest_pair


In [5]:
convert_lengths_to_floats(G)
# Separate the graph into components
wcc = [cc for cc in nx.weakly_connected_component_subgraphs(G)]
wcc.sort(key=len, reverse=True)

ds = []
for component in wcc:
    num_edges = len(component.edges())
    ds.append(calc_avg_directness(component, int(num_edges**0.7)))
ds

[0.7648613051718686,
 0.7343356315721234,
 0.7420760143971832,
 0.8191443347954969,
 0.7378963701383855,
 1.001983400981681,
 0.999673581596141,
 0.8250864604793381,
 1.0015956574417908,
 0.9073006056780458,
 0.9379728621193912,
 0.971378981875917,
 0.9128755104835539,
 0.9786185919522088,
 0.9973638081128271,
 0.9127434890429086,
 1.0025561528889038,
 0.9853078566996968,
 0.9719435223624527,
 0.9120387125516936,
 0.999187971294352,
 0.9899101985054352,
 0.8957981162977794,
 0.9639994925681419,
 0.9401330550558847,
 1.0026145061358769,
 1.0018674212828291,
 0.9850177649429644,
 0.8780106608539596,
 1.0017630438157192,
 0.9993691531407217,
 1.0011685096977487,
 0.9966030709164675,
 0.9184484656604768,
 0.9991520825920828,
 1.0016509200733466,
 0.997665821974729,
 0.9989337809871051,
 1.001701033350583,
 0.9975729460346459,
 0.9989245374708997,
 0.999683877398664,
 1.0026345574253273,
 1.0017498600654813,
 1.001595830538935,
 1.001730718529688,
 0.9997617701457238,
 1.002588203849152,
 1

In [53]:
def new_city(G, algy, length):
    """
    wcc = Sorted list of weakly connected subgraphs
    algy = desired path-adding algorithm
    
    """
    added_edges = []
    tot_length = []
    i = 0
    while sum(tot_length) < length:
        wcc = [cc for cc in nx.weakly_connected_component_subgraphs(G)]
        wcc.sort(key=len, reverse=True)
        added_edge = algy(wcc[i:])
        edge = (added_edge['i'],added_edge['j'],added_edge['dist'])
        G.add_edge(edge[0], edge[1], length = edge[2])
        added_edges.append(edge)
        tot_length.append(edge[2])
        i = i+1
    return G, added_edges, sum(tot_length)     

In [54]:
name = 'Amsterdam'
G = get_data(name)

G_new, edges, length = new_city(G, L2S, 35000)

KeyboardInterrupt: 

In [ ]:
ox.plot_graph(G_new)

In [52]:
sum(lengths)
edges

[(824620150, 174101627, 1207.129340637136),
 (1342036774, 30153896, 2745.060281655068),
 (123892541, 808367492, 34.04993383654771),
 (46565157, 5324097817, 4274.262644050204),
 (46515337, 46313604, 6738.97290886752),
 (1509163403, 286565233, 11064.33901629516),
 (46235775, 46467690, 6887.574948240108),
 (46467276, 1332094219, 3034.4038941095473)]